<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/05_crewAI/06_job_application_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Crew to Tailor Job Applications

In this notebook, we will built our first multi-agent system.

The libraries required are the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


In [ ]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- We can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Thought: I need to gather more information to create a comprehensive profile document for Noah.

Action: Search the internet
Action Input: {"search_query": "Noah software engineering leader experience managing remote and in-office teams programming languages AI data science tech initiatives startups innovation growth leadership roles strategic innovative approach"} 


Search results: Title: Noah Nemec - Software Developer IV | LinkedIn
Link: https://www.linkedin.com/in/noah-nemec-9a4469213
Snippet: I am a qualified and professional full stack software developer with two years of experience in desktop and mobile software development. Strong analytical and ...
---
Title: Noah Shapiro - Software Engineer at Microsoft | LinkedIn
Link: https://www.linkedin.com/in/shapiro-noah
Snippet: Educates and leads a team that develops virtual reality experiences in order to explore the boundaries for VR/AR media in a variety of fields.
---
Title: Summit 23 - AI for Good - ITU
Link: https://aiforgood.i

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is an accomplished Software Engineering Leader with 18 years of experience, specializing in managing remote and in-office teams. He holds an MBA and has a strong background in AI and data science. Noah has successfully led major tech initiatives and startups, proving his ability to drive innovation and growth in the tech industry.

## Skills
- Proficient in managing remote and in-office teams
- Expertise in multiple programming languages and frameworks
- Strong background in AI and data science
- Strategic thinker with innovative approach

## Work Experience
### Director of Software Engineering - DataKernel (2022 - Present)
- Transformed engineering division into a key revenue pillar
- Spearheaded integration of cutting-edge AI technologies
- Focused on achieving strategic project goals

### Senior Software Engineering Manager - DataKernel (2019 - 2022)
- Directed engine

Final Answer:

Interview Questions and Talking Points for Noah Williams:

1. Can you elaborate on your experience in managing remote and in-office teams? How do you ensure effective communication and collaboration across different time zones?
2. You mentioned expertise in multiple programming languages and frameworks. Could you provide examples of projects where you utilized this diverse skill set to deliver successful outcomes?
3. As someone with a strong background in AI and data science, how have you applied these skills to drive innovation and growth in your previous roles?
4. Could you share a specific example of a major tech initiative or startup that you led to success? What were the key challenges you faced, and how did you overcome them?
5. In your role as a Director of Software Engineering at DataKernel, how did you transform the engineering division into a key revenue pillar? What strategies did you implement to achieve this transformation?
6. As the Founder & CEO of InnovPe

- Dislplay the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is an accomplished Software Engineering Leader with 18 years of experience, specializing in managing remote and in-office teams. He holds an MBA and has a strong background in AI and data science. Noah has successfully led major tech initiatives and startups, proving his ability to drive innovation and growth in the tech industry.

## Skills
- Proficient in managing remote and in-office teams
- Expertise in multiple programming languages and frameworks
- Strong background in AI and data science
- Strategic thinker with innovative approach

## Work Experience
### Director of Software Engineering - DataKernel (2022 - Present)
- Transformed engineering division into a key revenue pillar
- Spearheaded integration of cutting-edge AI technologies
- Focused on achieving strategic project goals

### Senior Software Engineering Manager - DataKernel (2019 - 2022)
- Directed engineering strategy and operations
- Managed diverse teams across multiple time zones
- Recruited, mentored, and retained top talent

### Founder & CEO - InnovPet (2019 - 2022)
- Founded startup focused on innovative IoT solutions for pet care
- Oversaw product development from concept through execution
- Set up advisory board and established production facilities overseas

### Engineering Manager - EliteDevs (2018 - 2019)
- Formulated and executed strategic plans to enhance inter-departmental coordination
- Managed multiple engineering teams and implemented goal-setting frameworks

### Engineering Manager - PrintPack (2016 - 2018)
- Led development of high-performance engineering team
- Integrated data analytics into business decision-making processes

### Senior Software Engineer - DriveAI (2015 - 2016)
- Developed and optimized a central API
- Implemented critical enhancements, including advanced caching strategies

### CTO - BetCraft (2013 - 2015)
- Led technology strategy post-Series A funding
- Improved platform performance and expanded market reach

## Education
- MBA in Information Technology - London Business School
- Advanced Leadership Techniques Certification - University of London
- Data Science Specialization Certification - Coursera (Johns Hopkins University)
- B.Sc. in Computer Science - University of Edinburgh

Noah Williams is a highly skilled and experienced Software Engineering Leader with a proven track record of success in leading tech initiatives and startups. His strong background in AI, data science, and managing remote and in-office teams make him an ideal candidate for leadership roles that require innovation and strategic thinking.

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

Interview Questions and Talking Points for Noah Williams:

1. Can you elaborate on your experience in managing remote and in-office teams? How do you ensure effective communication and collaboration across different time zones?
2. You mentioned expertise in multiple programming languages and frameworks. Could you provide examples of projects where you utilized this diverse skill set to deliver successful outcomes?
3. As someone with a strong background in AI and data science, how have you applied these skills to drive innovation and growth in your previous roles?
4. Could you share a specific example of a major tech initiative or startup that you led to success? What were the key challenges you faced, and how did you overcome them?
5. In your role as a Director of Software Engineering at DataKernel, how did you transform the engineering division into a key revenue pillar? What strategies did you implement to achieve this transformation?
6. As the Founder & CEO of InnovPet, what were the key factors that contributed to the success of the startup? How did you navigate the initial funding phase and establish production facilities overseas?
7. Can you discuss a time when you actively mentored and developed team members? How do you approach mentoring to ensure the growth and success of your team?
8. Given your interest in exploring new technologies and trends, how do you stay updated with the latest advancements in the tech industry? How do you incorporate these trends into your work?
9. As a strategic thinker with an innovative approach, how do you balance short-term goals with long-term strategic initiatives in your leadership roles?
10. How do you effectively communicate complex technical concepts to non-technical stakeholders? Can you provide an example of a project where this skill was crucial to project success?

These questions and talking points aim to delve into Noah's experiences, skills, and achievements, allowing him to showcase his expertise and align his qualifications with the requirements of the job position.

## To develop more advanced multi-agent systems, go to [learn.crewai.com](https://learn.crewai.com).